# 1. Use “pip” to download the necessary libraries.

In [ ]:
!pip install datasets
! pip install -U accelerate
! pip install -U transformers
!pip install fugashi
!pip install ipadic

# 2. model train

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

with open('/content/drive/MyDrive/causality/label/label_doctor.jsonl', 'r', encoding='utf-8') as f:
    input_data = [json.loads(line) for line in f]

In [ ]:
from transformers import BertJapaneseTokenizer, BertForTokenClassification
from transformers import TrainingArguments
from transformers import Trainer
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import accelerate
import transformers
import json
import unicodedata
import os

transformers.__version__, accelerate.__version__

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
# MODEL_NAME = "izumi-lab/electra-base-japanese-discriminator"
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
# 定义关系标签到ID的映射
# label_map = {'Internal Cause': 1, 'External Cause': 2, 'Positive Effect': 3, 'Negative Effect': 4, 'Cues': 5}
model = BertForTokenClassification.from_pretrained(MODEL_NAME, num_labels=6)

# GPU使えるならGPU使う
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
# This code processes input NER-style data and converts it into a structured format with id, text, and a list of labeled entities (with position and type).

output_data = []

for item in input_data:
    output_item = {
        'id': str(item['id']),
        'text': item['text'],
        'entities': []
    }
    for label in item['label']:
        entity = {
            'span': [label[0], label[1]],
            'type': label[2]
        }
        output_item['entities'].append(entity)
    output_data.append(output_item)


In [ ]:
dataset = output_data
dataset[680]

{'id': '681',
 'text': '(4) 優先的に対処すべき事業上及び財務上の課題建設業界におきましては、大都市圏では今後も人口集中に伴うインフラ整備や再開発事業の加速が見込めること、公共インフラの防・減災、老朽化対策需要等が増加基調にあること、大規模自然災害の復旧需要が本格化することなどから一定程度の市場規模の維持は期待できますが、長期間に亘って新型コロナウイルス問題に起因する内外経済の減速が続けば、国内景気がこれまでどおりの拡大基調を維持することは期待できず、殊に地方圏の建設業界を取り巻く事業環境は楽観視できない状況になることは言を待ちません。',
 'entities': [{'span': [36, 49], 'type': 'External Cause'},
  {'span': [49, 52], 'type': 'Cues'},
  {'span': [52, 74], 'type': 'External Cause'},
  {'span': [75, 105], 'type': 'External Cause'},
  {'span': [106, 126], 'type': 'External Cause'},
  {'span': [128, 130], 'type': 'Cues'},
  {'span': [130, 149], 'type': 'Positive Effect'},
  {'span': [151, 169], 'type': 'External Cause'},
  {'span': [169, 174], 'type': 'Cues'},
  {'span': [174, 184], 'type': 'Negative Effect'},
  {'span': [186, 260], 'type': 'Negative Effect'}]}

In [ ]:
import random

# 固有表現のタイプとIDを対応付る辞書
type_id_dict = {
    "Internal Cause": 1,
    "External Cause": 2,
    "Positive Effect": 3,
    "Negative Effect": 4,
    "Cues": 5,
}

# カテゴリーをラベルに変更、文字列の正規化する。
for sample in dataset:
    sample['text'] = unicodedata.normalize('NFKC', sample['text'])
    for e in sample["entities"]:
        e['type_id'] = type_id_dict[e['type']]
        del e['type']

# データセットをシャッフル
random.shuffle(dataset)

# データセットの分割
n = len(dataset)
n_train = int(n*0.6)
n_val = int(n*0.2)
dataset_train = dataset[:n_train]
dataset_val = dataset[n_train:n_train+n_val]
dataset_test = dataset[n_train+n_val:]

print(f"Length of train: {len(dataset_train)}")
print(f"Length of val: {len(dataset_val)}")
print(f"Length of test: {len(dataset_test)}")

Length of train: 600
Length of val: 200
Length of test: 200


In [ ]:
class NerTokenizerForTrain(BertJapaneseTokenizer):

  def create_tokens_and_labels(self, splitted):
      """分割された文字列をトークン化し、ラベルを付与
      Args：
        splitted: 分割された文字列
          例：
          [{'text': 'レッドフォックス株式会社', 'label': 2},
          {'text': 'は、', 'label': 0},
          {'text': '東京都千代田区', 'label': 5},
          {'text': 'に本社を置くITサービス企業である。', 'label': 0}]
      Return:
        tokens, labels
          例：
          ['レッド', 'フォックス', '株式会社', 'は', '、', '東京', '都', '千代田', '区', 'に', '本社', 'を', '置く', 'IT', 'サービス', '企業', 'で', 'ある', '。']
          [2, 2, 2, 0, 0, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
      """
      tokens = [] # トークン格納用
      labels = [] # トークンに対応するラベル格納用
      for s in splitted:
          text = s['text']
          label = s['label']
          tokens_splitted = self.tokenize(text) # BertJapaneseTokenizerのトークナイザを使ってトークンに分割
          labels_splitted = [label] * len(tokens_splitted)
          tokens.extend(tokens_splitted)
          labels.extend(labels_splitted)

      return tokens, labels


  def encoding_for_bert(self, tokens, labels, max_length):
      """符号化を行いBERTに入力できる形式にする
      Args:
        tokens: トークン列
        labels: トークンに対応するラベルの列
      Returns:
        encoding: BERTに入力できる形式
        例：
        {'input_ids': [2, 3990, 13779, 1275, 9, 6, 391, 409, 9674, 280, 7, 2557, 11, 3045, 8267, 1645, 1189, 12, 31, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        　'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        　'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
          'labels': [0, 2, 2, 2, 0, 0, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

      """
      encoding = self.encode_plus(
          tokens,
          max_length=max_length,
          padding='max_length',
          truncation=True
      )
      # トークン[CLS]、[SEP]のラベルを0
      labels = [0] + labels[:max_length-2] + [0]
      # トークン[PAD]のラベルを0
      labels = labels + [0]*( max_length - len(labels) )
      encoding['labels'] = labels

      return encoding


  def encode_plus_tagged(self, text, entities, max_length):
      """文章とそれに含まれる固有表現が与えられた時に、符号化とラベル列の作成
      Args:
        text: 元の文章
        entities: 文章中の固有表現の位置(span)とラベル(type_id)の情報

      """
      # 固有表現の前後でtextを分割し、それぞれのラベルをつけておく。
      entities = sorted(entities, key=lambda x: x['span'][0]) # 固有表現の位置の昇順でソート
      splitted = [] # 分割後の文字列格納用
      position = 0
      for entity in entities:
          start = entity['span'][0]
          end = entity['span'][1]
          label = entity['type_id']
          # 固有表現ではないものには0のラベルを付与
          splitted.append({'text': text[position:start], 'label':0})
          # 固有表現には、固有表現のタイプに対応するIDをラベルとして付与
          splitted.append({'text': text[start:end], 'label':label})
          position = end

      # 最後の固有表現から文末に、0のラベルを付与
      splitted.append({'text': text[position:], 'label':0})
      # positionとspan[0]の値が同じだと空白文字にラベル0が付与されるため、長さ0の文字列は除く（例：{'text': '', 'label': 0}）
      splitted = [ s for s in splitted if s['text'] ]

      # 分割された文字列をトークン化し、ラベルを付与
      tokens, labels = self.create_tokens_and_labels(splitted)

      # 符号化を行いBERTに入力できる形式にする
      encoding = self.encoding_for_bert(tokens, labels, max_length)

      return encoding

In [ ]:
class NerTokenizerForTest(BertJapaneseTokenizer):

    def encoding_for_bert(self, tokens, max_length):
        """符号化を行いBERTに入力できる形式にする
        Args:
          tokens: トークン列
        Returns:
          encoding: BERTに入力できる形式
          例：
          {'input_ids': [2, 106, 6, 946, 674, 5, 12470, 9921, 5, 859, 6, 2446, 22903, 35, 24831, 11614, 35, 2176, 2200, 35, 3700, 29650, 2446, 333, 9, 6, 2409, 109, 5, 333, 3849, 3],
          'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
        """
        encoding = self.encode_plus(
            tokens,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_tensors = "pt"
        )

        return encoding


    def create_spans_of_token(self, tokens_original, encoding):
        """ 各トークン（サブワード）の文章中での位置を調べる
          Args:
            tokens_original: トークン列をさらにサブワードに分割した列
            encoding:
            例：tokens_original
              ['元々', 'は', '前作', '「', 'The', 'Apple', 's', '」', 'の', 'アウト', ...]

          Return:
            spans: 各トークンの文章中の位置([CLS][PAD]などの特殊トークンはダミーで置き換える)
            例：
              [[-1, -1], [0, 2], [2, 3], [3, 5], [5, 6], [6, 9], [10, 15], [15, 16], ...]
        """
        position = 0
        spans = [] # トークンの位置を追加していく。
        for token in tokens_original:
            l = len(token)
            while 1:
                if token != text[position:position+l]:
                    """例：英語文章のように空白が混ざっていると下記のようにずれるケースがあることを考慮
                          token: "Digital"
                          text[position:position+l]: " Digita"
                    """
                    position += 1
                else:
                    spans.append([position, position+l])
                    position += l
                    break

        sequence_length = len(encoding['input_ids'])
        # 特殊トークン[CLS]に対するダミーのspanを追加。
        spans = [[-1, -1]] + spans[:sequence_length-2]
        # 特殊トークン[SEP]、[PAD]に対するダミーのspanを追加。
        spans = spans + [[-1, -1]] * ( sequence_length - len(spans) )

        return spans


    def encode_plus_untagged(self, text, max_length=None):
        """文章をトークン化し、それぞれのトークンの文章中の位置も特定しておく。
        """
        # 文章のトークン化を行い、
        # それぞれのトークンと文章中の文字列を対応づける。
        tokens = [] # トークン格納用
        tokens_original = [] # トークンに対応する文章中の文字列格納用
        words = self.word_tokenizer.tokenize(text) # MeCabで単語に分割
        for word in words:
            # 単語をサブワードに分割
            tokens_word = self.subword_tokenizer.tokenize(word)
            tokens.extend(tokens_word)
            if tokens_word[0] == '[UNK]': # 未知語への対応
                tokens_original.append(word)
            else:
                tokens_original.extend([
                    token.replace('##','') for token in tokens_word
                ])


        # 符号化を行いBERTに入力できる形式にする
        encoding = self.encoding_for_bert(tokens, max_length)

        # 各トークン（サブワード）の文章中での位置を調べる
        spans = self.create_spans_of_token(tokens_original, encoding)

        return encoding, spans


    def convert_bert_output_to_entities(self, text, labels, spans):
        """文章、ラベル列の予測値、各トークンの位置から固有表現を得る。
        """
        # labels, spansから特殊トークンに対応する部分を取り除く
        labels = [label for label, span in zip(labels, spans) if span[0] != -1]
        spans = [span for span in spans if span[0] != -1]

        # 同じラベルが連続するトークンをまとめて、固有表現を抽出する。
        entities = []
        position = 0
        for label, group in itertools.groupby(labels):
            """
            例：labelsは予測結果
            labels: [0, 0, 0, 3, 3, 5, 7, 7, 7, 0, 0, 0]
            """
            start_idx = position # 連続するラベルの先頭位置
            end_idx = position + len(list(group)) - 1 # 連続するラベルの最終位置

            # (encode_plus_untaggedで計算した)spansから、文章中の位置を特定
            start = spans[start_idx][0]
            end = spans[end_idx][1]

            # 次のspanの位置に更新
            position = end_idx + 1

            if label != 0: # ラベルが0以外ならば、新たな固有表現として追加。
                entity = {
                    "name": text[start:end],
                    "span": [start, end],
                    "type_id": label
                }
                entities.append(entity)

        return entities

In [ ]:
class CreateDataset(Dataset):
  """データセット作成
  """
  def __init__(self, dataset, tokenizer, max_length):
    self.dataset = dataset
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, index):
    text = self.dataset[index]["text"]
    entities = self.dataset[index]["entities"]
    encoding = tokenizer.encode_plus_tagged(text, entities, max_length=self.max_length)

    input_ids = torch.tensor(encoding["input_ids"])
    token_type_ids = torch.tensor(encoding["token_type_ids"])
    attention_mask = torch.tensor(encoding["attention_mask"])
    labels = torch.tensor(encoding["labels"])

    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_mask,
      "labels": labels
    }


In [ ]:
# 訓練時に使うトークナイザーをロード
tokenizer = NerTokenizerForTrain.from_pretrained(MODEL_NAME)

In [ ]:
import pprint
tmp = dataset_train[2]
pprint.pprint(tmp)
pprint.pprint(tokenizer.encode_plus_tagged(text=tmp["text"], entities=tmp["entities"], max_length=32), width=200)

{'entities': [{'span': [3, 10], 'type_id': 1},
              {'span': [10, 13], 'type_id': 5},
              {'span': [13, 21], 'type_id': 1},
              {'span': [23, 37], 'type_id': 1},
              {'span': [38, 40], 'type_id': 5},
              {'span': [41, 52], 'type_id': 1},
              {'span': [54, 65], 'type_id': 1},
              {'span': [67, 74], 'type_id': 1}],
 'id': '852',
 'text': 'また、データ量の増加による回線負荷への対応や、有事の際のサービス継続性強化のため、サーバー及び回線の増強や、バックアップ体制の強化等、運用保守の改善に努めていきます。'}
{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [2, 106, 6, 1676, 1073, 5, 2157, 250, 11994, 12473, 118, 5, 1277, 49, 6, 26919, 5, 596, 5, 1645, 2934, 245, 2808, 5, 82, 6, 14979, 920, 11994, 5, 9579, 3],
 'labels': [0, 0, 0, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 5, 0, 1, 1, 1, 1, 1, 0],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# データセットの作成
dataset_train_for_loader = CreateDataset(dataset_train, tokenizer, max_length=512)
dataset_val_for_loader = CreateDataset(dataset_val, tokenizer, max_length=512)
dataset_test_for_loader = CreateDataset(dataset_test, tokenizer, max_length=512)

# データローダーの作成
dataloader_train = DataLoader(dataset_train_for_loader, batch_size=32, shuffle=True, pin_memory=True)
dataloader_val = DataLoader(dataset_val_for_loader, batch_size=256, shuffle=True, pin_memory=True)
dataloader_test = DataLoader(dataset_test_for_loader, batch_size=256, shuffle=True, pin_memory=True)

dataloaders_dict = {"train": dataloader_train, "val": dataloader_val, "test": dataloader_test}

In [ ]:
# 最適化器
optimizer = torch.optim.Adam(params=model.parameters(), lr=2e-5)

In [ ]:
# モデルを学習させる関数を作成
def train_model(net, dataloaders_dict, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    loss, logits = model(input_ids=input_ids,
                                          token_type_ids=None,
                                          attention_mask=attention_mask,
                                          labels=labels,
                                          return_dict=False)

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                        optimizer.step()

                        # if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                        #     print(f"イテレーション {iteration} || Loss: {loss:.4f}")

                    iteration += 1

                    # 損失の合計を更新
                    epoch_loss += loss.item() * batch_size

            # epochごとのloss
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)

            print(f"Epoch {epoch+1}/{num_epochs} | phase {phase} |  Loss: {epoch_loss:.4f}")

    return net


In [ ]:
# 学習・検証を実行
# 訓練時に使うトークナイザーをロード
# tokenizer = NerTokenizerForTrain.from_pretrained(MODEL_NAME)
num_epochs = 15
net_trained = train_model(model, dataloaders_dict, optimizer, num_epochs=num_epochs)

In [ ]:
# テスト時に使うトークナイザーをロード
tokenizer = NerTokenizerForTest.from_pretrained(MODEL_NAME)

In [ ]:
from tqdm import tqdm
import os
import json
import unicodedata
import itertools

def predict(text, tokenizer, model):
    """BERTで固有表現抽出を行うための関数。
    """
    # 符号化
    encoding, spans = tokenizer.encode_plus_untagged(text)
    encoding = { k: v.cuda() for k, v in encoding.items() }

    # ラベルの予測値の計算
    with torch.no_grad():
        output = model(**encoding)
        scores = output.logits
        labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist()

    # ラベル列を固有表現に変換
    entities = tokenizer.convert_bert_output_to_entities(
        text, labels_predicted, spans
    )

    return entities

# 固有表現抽出
entities_list = [] # 正解の固有表現
entities_predicted_list = [] # 予測された固有表現
for sample in tqdm(dataset_test):
    text = sample['text']
    entities_predicted = predict(text, tokenizer, net_trained) # BERTで予測
    entities_list.append(sample['entities'])
    entities_predicted_list.append( entities_predicted )

100%|██████████| 200/200 [00:06<00:00, 29.17it/s]


In [ ]:
i = 1
print("# 正解 #")
print(entities_list[i])
print("# 推論 #")
print(entities_predicted_list[i])
print("# もとの文章 #")
print(dataset_test[i]["text"])

# 正解 #
[{'span': [19, 24], 'type_id': 2}, {'span': [25, 29], 'type_id': 2}, {'span': [29, 33], 'type_id': 5}, {'span': [34, 49], 'type_id': 2}, {'span': [49, 51], 'type_id': 5}, {'span': [51, 69], 'type_id': 4}, {'span': [80, 103], 'type_id': 3}]
# 推論 #
[{'name': '少子高齢化', 'span': [19, 24], 'type_id': 2}, {'name': '人口減少', 'span': [25, 29], 'type_id': 2}, {'name': 'に加えて', 'span': [29, 33], 'type_id': 5}, {'name': '燃費改善や燃料転換の構造的要因', 'span': [34, 49], 'type_id': 2}, {'name': 'から', 'span': [49, 51], 'type_id': 5}, {'name': '燃料油の国内需要は減少傾向', 'span': [51, 64], 'type_id': 4}, {'name': '継続', 'span': [65, 67], 'type_id': 4}, {'name': '石油製品の需要増加', 'span': [94, 103], 'type_id': 3}]
# もとの文章 #
石油業界を取り巻く環境につきましては、少子高齢化や人口減少に加えて、燃費改善や燃料転換の構造的要因から燃料油の国内需要は減少傾向が継続するものと予想されますが、世界的にはアジア諸国を中心に石油製品の需要増加が見込まれます。


In [ ]:
def evaluate_model(entities_list, entities_predicted_list, type_id=None):
    num_entities = 0
    num_predictions = 0
    num_correct = 0
    num_correct_pred = 0

    for entities, entities_predicted in zip(entities_list, entities_predicted_list):
        if type_id:
            entities = [e for e in entities if e['type_id'] == type_id]
            entities_predicted = [e for e in entities_predicted if e['type_id'] == type_id]

        num_entities += len(entities)
        num_predictions += len(entities_predicted)

        # Calculate num_correct (for recall)
        for e in entities:
            span = e['span']
            for e_pred in entities_predicted:
                span_pred = e_pred['span']
                if (span[0] <= span_pred[1] and span_pred[0] <= span[1]) and e['type_id'] == e_pred['type_id']:
                    num_correct += 1
                    break

        # Calculate num_correct_pred (for precision)
        for e_pred in entities_predicted:
            span_pred = e_pred['span']
            for e in entities:
                span = e['span']
                if (span_pred[0] <= span[1] and span[0] <= span_pred[1]) and e_pred['type_id'] == e['type_id']:
                    num_correct_pred += 1
                    break

    precision = num_correct_pred / num_predictions if num_predictions > 0 else 0
    recall = num_correct / num_entities if num_entities > 0 else 0
    f_value = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

    result = {
        'num_entities': num_entities,
        'num_predictions': num_predictions,
        'num_correct': num_correct,
        'num_correct_pred': num_correct_pred,
        'precision': precision,
        'recall': recall,
        'f_value': f_value
    }

    return result

In [ ]:
# 評価結果 Evaluate the results
import pandas as pd
eval_df = pd.DataFrame()
for k, v in type_id_dict.items():
  eval_res = evaluate_model(entities_list, entities_predicted_list, type_id=v)
  eval_df[k] = eval_res.values()

eval_res_all = evaluate_model(entities_list, entities_predicted_list, type_id=None)
eval_df["ALL"] = eval_res_all.values()

eval_df.index = eval_res_all.keys()
eval_df

,Internal Cause,External Cause,Positive Effect,Negative Effect,Cues,ALL
num_entities,209.000000,264.000000,219.000000,135.000000,311.000000,1138.000000
num_predictions,330.000000,306.000000,312.000000,233.000000,392.000000,1573.000000
num_correct,169.000000,228.000000,180.000000,123.000000,285.000000,985.000000
num_correct_pred,221.000000,247.000000,212.000000,142.000000,284.000000,1106.000000
precision,0.669697,0.807190,0.679487,0.609442,0.724490,0.703115
recall,0.808612,0.863636,0.821918,0.911111,0.916399,0.865554
f_value,0.732628,0.834459,0.743947,0.730352,0.809222,0.775924


In [ ]:
save_path = "/content/drive/MyDrive/causality/label/causal_model.pth"
torch.save(net_trained, save_path)

In [ ]:
text = "基本方針当社グループを取り巻く事業環境は、世界的には新興国を中心に自動車需要や鉄鋼需要の拡大が期待される一方で、人口の減少や高齢化の進展等により国内需要の拡大は期待できないと見込まれます。"
entities_predicted = predict(text, tokenizer, net_trained)
entities_predicted

[{'name': '自動車需要や鉄鋼需要の拡大が期待される', 'span': [33, 52], 'type_id': 2},
 {'name': '一方で', 'span': [52, 55], 'type_id': 5},
 {'name': '人口の減少', 'span': [56, 61], 'type_id': 2},
 {'name': '高齢化の進展', 'span': [62, 68], 'type_id': 2},
 {'name': 'により', 'span': [69, 72], 'type_id': 5},
 {'name': '国内需要の拡大は期待できないと', 'span': [72, 87], 'type_id': 4},
 {'name': 'ます', 'span': [91, 93], 'type_id': 4}]

# 3. Calculate SAB for all documents

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
! pip install -U accelerate
! pip install -U transformers
!pip install fugashi
!pip install ipadic
!pip install unidic_lite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 130.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import BertJapaneseTokenizer, BertForTokenClassification
from transformers import TrainingArguments
from transformers import Trainer
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import accelerate
import transformers
import json
import unicodedata
import os

transformers.__version__, accelerate.__version__

In [ ]:
load_path = "/content/drive/MyDrive/causality/label/causal_model.pth"
model = torch.load(load_path)

# GPU使えるならGPU使う
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

<ipython-input-5-833fade5ffa2>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(load_path)


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
class NerTokenizerForTest(BertJapaneseTokenizer):

    def encoding_for_bert(self, tokens, max_length):
        """符号化を行いBERTに入力できる形式にする
        Args:
          tokens: トークン列
        Returns:
          encoding: BERTに入力できる形式
          例：
          {'input_ids': [2, 106, 6, 946, 674, 5, 12470, 9921, 5, 859, 6, 2446, 22903, 35, 24831, 11614, 35, 2176, 2200, 35, 3700, 29650, 2446, 333, 9, 6, 2409, 109, 5, 333, 3849, 3],
          'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
        """
        encoding = self.encode_plus(
            tokens,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_tensors = "pt"
        )

        return encoding


    def create_spans_of_token(self, tokens_original, encoding):
        """ 各トークン（サブワード）の文章中での位置を調べる
          Args:
            tokens_original: トークン列をさらにサブワードに分割した列
            encoding:
            例：tokens_original
              ['元々', 'は', '前作', '「', 'The', 'Apple', 's', '」', 'の', 'アウト', ...]

          Return:
            spans: 各トークンの文章中の位置([CLS][PAD]などの特殊トークンはダミーで置き換える)
            例：
              [[-1, -1], [0, 2], [2, 3], [3, 5], [5, 6], [6, 9], [10, 15], [15, 16], ...]
        """
        position = 0
        spans = [] # トークンの位置を追加していく。
        for token in tokens_original:
            l = len(token)
            while 1:
                if token != text[position:position+l]:
                    """例：英語文章のように空白が混ざっていると下記のようにずれるケースがあることを考慮
                          token: "Digital"
                          text[position:position+l]: " Digita"
                    """
                    position += 1
                else:
                    spans.append([position, position+l])
                    position += l
                    break

        sequence_length = len(encoding['input_ids'])
        # 特殊トークン[CLS]に対するダミーのspanを追加。
        spans = [[-1, -1]] + spans[:sequence_length-2]
        # 特殊トークン[SEP]、[PAD]に対するダミーのspanを追加。
        spans = spans + [[-1, -1]] * ( sequence_length - len(spans) )

        return spans


    def encode_plus_untagged(self, text, max_length=None):
        """文章をトークン化し、それぞれのトークンの文章中の位置も特定しておく。
        """
        # 文章のトークン化を行い、
        # それぞれのトークンと文章中の文字列を対応づける。
        tokens = [] # トークン格納用
        tokens_original = [] # トークンに対応する文章中の文字列格納用
        words = self.word_tokenizer.tokenize(text) # MeCabで単語に分割
        for word in words:
            # 単語をサブワードに分割
            tokens_word = self.subword_tokenizer.tokenize(word)
            tokens.extend(tokens_word)
            if tokens_word[0] == '[UNK]': # 未知語への対応
                tokens_original.append(word)
            else:
                tokens_original.extend([
                    token.replace('##','') for token in tokens_word
                ])


        # 符号化を行いBERTに入力できる形式にする
        encoding = self.encoding_for_bert(tokens, max_length)

        # 各トークン（サブワード）の文章中での位置を調べる
        spans = self.create_spans_of_token(tokens_original, encoding)

        return encoding, spans


    def convert_bert_output_to_entities(self, text, labels, spans):
        """文章、ラベル列の予測値、各トークンの位置から固有表現を得る。
        """
        # labels, spansから特殊トークンに対応する部分を取り除く
        labels = [label for label, span in zip(labels, spans) if span[0] != -1]
        spans = [span for span in spans if span[0] != -1]

        # 同じラベルが連続するトークンをまとめて、固有表現を抽出する。
        entities = []
        position = 0
        for label, group in itertools.groupby(labels):
            """
            例：labelsは予測結果
            labels: [0, 0, 0, 3, 3, 5, 7, 7, 7, 0, 0, 0]
            """
            start_idx = position # 連続するラベルの先頭位置
            end_idx = position + len(list(group)) - 1 # 連続するラベルの最終位置

            # (encode_plus_untaggedで計算した)spansから、文章中の位置を特定
            start = spans[start_idx][0]
            end = spans[end_idx][1]

            # 次のspanの位置に更新
            position = end_idx + 1

            if label != 0: # ラベルが0以外ならば、新たな固有表現として追加。
                entity = {
                    "name": text[start:end],
                    "span": [start, end],
                    "type_id": label
                }
                entities.append(entity)

        return entities

In [ ]:
tokenizer = NerTokenizerForTest.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NerTokenizerForTest'.


In [ ]:
from tqdm import tqdm
import os
import json
import unicodedata
import itertools

def predict(text, tokenizer, model):
    """BERTで固有表現抽出を行うための関数。
    """
    # 符号化
    encoding, spans = tokenizer.encode_plus_untagged(text)
    encoding = { k: v.cuda() for k, v in encoding.items() }

    # ラベルの予測値の計算
    with torch.no_grad():
        output = model(**encoding)
        scores = output.logits
        labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist()

    # ラベル列を固有表現に変換
    entities = tokenizer.convert_bert_output_to_entities(
        text, labels_predicted, spans
    )

    return entities



In [ ]:
text = "基本方針当社グループを取り巻く事業環境は、世界的には新興国を中心に自動車需要や鉄鋼需要の拡大が期待される一方で、人口の減少や高齢化の進展等により国内需要の拡大は期待できないと見込まれます。"
entities_predicted = predict(text, tokenizer, model)
entities_predicted

[{'name': '自動車需要や鉄鋼需要の拡大が期待される', 'span': [33, 52], 'type_id': 2},
 {'name': '一方で', 'span': [52, 55], 'type_id': 5},
 {'name': '人口の減少や高齢化の進展', 'span': [56, 68], 'type_id': 2},
 {'name': 'に', 'span': [69, 70], 'type_id': 5},
 {'name': '国内需要の拡大は期待できない', 'span': [72, 86], 'type_id': 4}]

In [ ]:
import pandas as pd
df = pd.read_excel("/content/drive/MyDrive/causality/annualreport_text.xlsx")
df.astype(str)
df

,offer data,id,mda
0,20180705,1446,7 【財政状態、経営成績及びキャッシュ・フローの状況の分析】文中の将来に関する事項は、本書提...
1,20181213,1449,7【財政状態、経営成績及びキャッシュ・フローの状況の分析】文中の将来に関する事項は、本書提出...
2,20181218,1450,3 【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】(1) 経営成績等の...
3,20181217,2970,7【財政状態、経営成績及びキャッシュ・フローの状況の分析】文中の将来に関する事項は、本書提出...
4,20180228,3446,7【財政状態、経営成績及びキャッシュ・フローの状況の分析】文中の将来に関する事項は、本書提出...
...,...,...,...
480,20220928,9561,3 【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】(1) 経営成績等の...
481,20221020,9562,3 【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】(1)経営成績等の状...
482,20221026,9563,3 【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】(1) 経営成績等の...
483,20221027,9564,3 【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】(1) 経営成績等の...


In [ ]:
import re

def cut_sentences(content):

    end_flag = ['?', '!', '.', '？', '！', '。', '…']
    sentences = []


    chinese_punctuation = ['。', '！', '？', '…']


    pattern = re.compile(r"([{}])".format("".join(chinese_punctuation)))
    parts = pattern.split(content)
    tmp = ''

    for part in parts:
        if part in chinese_punctuation:
            sentences.append(tmp + part)
            tmp = ''
        else:
            tmp += part


    for idx, sentence in enumerate(sentences):
        if sentence.endswith('…') and idx < len(sentences) - 1:
            sentences[idx] += sentences[idx + 1]
            sentences[idx + 1] = ''


    sentences = [sentence.strip() for sentence in sentences if sentence.strip() and len(sentence) >= 30]

    return sentences

In [ ]:
def process_dataframe(df_test):

    processed_data = []


    for index, row in df_test.iterrows():
        date = row["offer data"]
        document_id = row["id"]
        document = row["mda"]


        sentence_lst = cut_sentences(document)


        for sentence in sentence_lst:
            processed_data.append({
                'date': date,
                'id': document_id,
                'sentence': sentence
            })


    processed_df = pd.DataFrame(processed_data)

    return processed_df


result_df = process_dataframe(df)
result_df

,date,id,sentence
0,20180705,1446,7 【財政状態、経営成績及びキャッシュ・フローの状況の分析】文中の将来に関する事項は、本書提...
1,20180705,1446,(1) 重要な会計方針及び見積り当社グループの連結財務諸表は、我が国において一般に公正妥当と...
2,20180705,1446,なお、この連結財務諸表の作成にあたっては、資産・負債および収益・費用に影響を与える見積りを必...
3,20180705,1446,これらの見積りにつきましては、経営者が過去の実績や取引状況を勘案し、会計基準の範囲内でかつ合...
4,20180705,1446,(2) 経営成績の分析第4期連結会計年度(自 平成28年10月1日 至 平成29年9月30日...
...,...,...,...
45360,20221130,9565,売上高は当社グループの成長性、売上高営業利益率はその成長の持続可能性を測る目安として重要視し...
45361,20221130,9565,指標第5期事業年度(実績)第6期事業年度(実績)第7期事業年度(計画)売上高 831百万円 ...
45362,20221130,9565,また、eスポーツ市場の拡大に伴い、eスポーツ選手・実況者・解説者・インフルエンサーの活躍の機...
45363,20221130,9565,売上高営業利益は、合併による人件費増加等がある一方、売上高の増加に伴い上昇傾向にあります。


In [ ]:
text_normalization = []
for sample in result_df['sentence']:
    sample= unicodedata.normalize('NFKC', sample)
    text_normalization.append(sample)

len(text_normalization)

45365

In [ ]:
entity_extraction = []
for text in tqdm(text_normalization):
    entity_pre = predict(text, tokenizer, model)
    entity_extraction.append(entity_pre)

len(entity_extraction)

100%|██████████| 45365/45365 [10:29<00:00, 72.02it/s]


45365

In [ ]:
result_df["entity_extraction"] = entity_extraction
result_df

,date,id,sentence,entity_extraction
0,20180705,1446,7 【財政状態、経営成績及びキャッシュ・フローの状況の分析】文中の将来に関する事項は、本書提...,[]
1,20180705,1446,(1) 重要な会計方針及び見積り当社グループの連結財務諸表は、我が国において一般に公正妥当と...,[]
2,20180705,1446,なお、この連結財務諸表の作成にあたっては、資産・負債および収益・費用に影響を与える見積りを必...,"[{'name': '資産・負債および収益・費用に影響を与える見積り', 'span': [..."
3,20180705,1446,これらの見積りにつきましては、経営者が過去の実績や取引状況を勘案し、会計基準の範囲内でかつ合...,"[{'name': '経営者が過去の実績や取引状況を勘案し', 'span': [15, 3..."
4,20180705,1446,(2) 経営成績の分析第4期連結会計年度(自 平成28年10月1日 至 平成29年9月30日...,"[{'name': '貸家の好調', 'span': [60, 65], 'type_id'..."
...,...,...,...,...
45360,20221130,9565,売上高は当社グループの成長性、売上高営業利益率はその成長の持続可能性を測る目安として重要視し...,"[{'name': 'その成長の持続可能性', 'span': [24, 34], 'typ..."
45361,20221130,9565,指標第5期事業年度(実績)第6期事業年度(実績)第7期事業年度(計画)売上高 831百万円 ...,"[{'name': 'eスポーツ市場の堅調な成長', 'span': [91, 104], ..."
45362,20221130,9565,また、eスポーツ市場の拡大に伴い、eスポーツ選手・実況者・解説者・インフルエンサーの活躍の機...,"[{'name': 'eスポーツ市場の拡大', 'span': [3, 13], 'type..."
45363,20221130,9565,売上高営業利益は、合併による人件費増加等がある一方、売上高の増加に伴い上昇傾向にあります。,"[{'name': '合併', 'span': [9, 11], 'type_id': 1}..."


In [ ]:
df = result_df

In [ ]:
def count_labels(entities):
  internal_cause = 0
  external_cause = 0
  positive_effect = 0
  negative_effect = 0
  cues = 0
  for entity in entities:
      label = entity['type_id']
      if label == 1:
          internal_cause += 1
      elif label == 2:
          external_cause += 1
      elif label == 3:
          positive_effect += 1
      elif label == 4:
          negative_effect += 1
      elif label == 5:
          cues += 1
  return internal_cause, external_cause, positive_effect, negative_effect, cues


count_internal_cause = []
count_external_cause = []
count_positive_effect = []
count_negative_effect = []
count_cues = []

for entity in entity_extraction:
    # print(entity)
    internal_cause, external_cause, positive_effect, negative_effect, cues = count_labels(entity)
    count_internal_cause.append(internal_cause)
    # print(count_internal_cause)
    count_external_cause.append(external_cause)
    count_positive_effect.append(positive_effect)
    count_negative_effect.append(negative_effect)
    count_cues.append(cues)


In [ ]:
# df["entity_extraction"] = entity_extraction
df["internal_cause"] = count_internal_cause
df["external_cause"] = count_external_cause
df["positive_effect"] = count_positive_effect
df["negative_effect"] = count_negative_effect
df["cues"] = count_cues

df

,date,id,sentence,entity_extraction,internal_cause,external_cause,positive_effect,negative_effect,cues
0,20180705,1446,7 【財政状態、経営成績及びキャッシュ・フローの状況の分析】文中の将来に関する事項は、本書提...,[],0,0,0,0,0
1,20180705,1446,(1) 重要な会計方針及び見積り当社グループの連結財務諸表は、我が国において一般に公正妥当と...,[],0,0,0,0,0
2,20180705,1446,なお、この連結財務諸表の作成にあたっては、資産・負債および収益・費用に影響を与える見積りを必...,"[{'name': '資産・負債および収益・費用に影響を与える見積り', 'span': [...",2,0,0,0,0
3,20180705,1446,これらの見積りにつきましては、経営者が過去の実績や取引状況を勘案し、会計基準の範囲内でかつ合...,"[{'name': '経営者が過去の実績や取引状況を勘案し', 'span': [15, 3...",4,0,0,4,1
4,20180705,1446,(2) 経営成績の分析第4期連結会計年度(自 平成28年10月1日 至 平成29年9月30日...,"[{'name': '貸家の好調', 'span': [60, 65], 'type_id'...",1,3,4,0,3
...,...,...,...,...,...,...,...,...,...
45360,20221130,9565,売上高は当社グループの成長性、売上高営業利益率はその成長の持続可能性を測る目安として重要視し...,"[{'name': 'その成長の持続可能性', 'span': [24, 34], 'typ...",1,0,1,0,0
45361,20221130,9565,指標第5期事業年度(実績)第6期事業年度(実績)第7期事業年度(計画)売上高 831百万円 ...,"[{'name': 'eスポーツ市場の堅調な成長', 'span': [91, 104], ...",2,1,1,0,3
45362,20221130,9565,また、eスポーツ市場の拡大に伴い、eスポーツ選手・実況者・解説者・インフルエンサーの活躍の機...,"[{'name': 'eスポーツ市場の拡大', 'span': [3, 13], 'type...",0,1,3,1,1
45363,20221130,9565,売上高営業利益は、合併による人件費増加等がある一方、売上高の増加に伴い上昇傾向にあります。,"[{'name': '合併', 'span': [9, 11], 'type_id': 1}...",2,0,2,0,3


In [ ]:
# save your results
df.to_excel("/content/drive/MyDrive/causality/result.xlsx")